In [29]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

In [74]:
data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
data.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [80]:
data = data.sample(frac=1)
data = data.reset_index(drop=True)
data.head()

,image_id,label
0,214682484.jpg,4
1,1527122136.jpg,3
2,3058897297.jpg,4
3,28277357.jpg,3
4,4050750401.jpg,3


In [81]:
extra_path = "../input/cassava-leaf-disease-classification/train_images/"
data["image_id"] = extra_path + data["image_id"]

In [82]:
data.head()

,image_id,label
0,../input/cassava-leaf-disease-classification/t...,4
1,../input/cassava-leaf-disease-classification/t...,3
2,../input/cassava-leaf-disease-classification/t...,4
3,../input/cassava-leaf-disease-classification/t...,3
4,../input/cassava-leaf-disease-classification/t...,3


In [83]:
type(data["label"][0])

4

In [99]:
# converting numpy.int --> str for categorization
convert_dict = { 'label': 'string'} 
data = data.astype(convert_dict)

In [ ]:
type(data["label"][0])

In [100]:
data.head()

,image_id,label
0,../input/cassava-leaf-disease-classification/t...,4
1,../input/cassava-leaf-disease-classification/t...,3
2,../input/cassava-leaf-disease-classification/t...,4
3,../input/cassava-leaf-disease-classification/t...,3
4,../input/cassava-leaf-disease-classification/t...,3


In [102]:
datagen=ImageDataGenerator(rescale=1./255)

In [103]:
train_df = data[:int(0.8*len(data))]
valid_df = data[int(0.8*len(data)):]

In [105]:
train_generator=datagen.flow_from_dataframe(dataframe=train_df, 
                                            x_col="image_id", y_col="label",
                                            class_mode="categorical", 
                                            target_size=(224,224), batch_size=32,
                                            shuffle=True)
valid_generator=datagen.flow_from_dataframe(dataframe=valid_df, 
                                            x_col="image_id", y_col="label",
                                            class_mode="categorical", 
                                            target_size=(224,224), batch_size=32)

Found 17117 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


In [106]:
# Build model
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(224, 224, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
 11/534 [..............................] - ETA: 11:05 - loss: 1.6871 - accuracy: 0.4858